In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Add, Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout

# import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras import optimizers

from tqdm import tqdm_notebook
from tensorflow.compat.v1.metrics import mean_iou

Using TensorFlow backend.


In [0]:
location = "C://Users//oana.sabadas//PycharmProjects//TGS//TGS//data//101"

In [0]:
y_train = np.load("{}//y_train.npy".format(location),allow_pickle=True)
x_train = np.load("{}//x_train.npy".format(location),allow_pickle=True)


y_valid = np.load("{}//y_valid.npy".format(location),allow_pickle=True)
x_valid = np.load("{}//x_valid.npy".format(location),allow_pickle=True)


ids_train = np.load("{}//ids_train.npy".format(location),allow_pickle=True)
ids_valid = np.load("{}//ids_valid.npy".format(location),allow_pickle=True)


cov_train = np.load("{}//cov_train.npy".format(location),allow_pickle=True)
cov_valid = np.load("{}//cov_test.npy".format(location),allow_pickle=True)


depth_train = np.load("{}//depth_train.npy".format(location),allow_pickle=True)
depth_valid = np.load("{}//depth_test.npy".format(location),allow_pickle=True)


In [0]:
x_train.shape

(3200, 101, 101, 1)

In [0]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [0]:
# Build model
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    
    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    
    # 12 -> 25
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)

    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    
    # 50 -> 101
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv1)
    output_layer =  Activation('sigmoid')(output_layer_noActi)
    
    return output_layer

      # Define IoU metric
      def mean_iou(y_true, y_pred):
          prec = []
          for t in np.arange(0.5, 1.0, 0.05):
          y_pred_ = tf.to_int32(y_pred > t)
          score, up_opt = mean_iou(y_true, y_pred_, 2)
          K.get_session().run(tf.local_variables_initializer())
          with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
          prec.append(score)
      return K.mean(K.stack(prec), axis=0)

In [0]:
# model
input_layer = Input((101, 101, 1))
output_layer = build_model(input_layer, 16,0.5)

model1 = Model(input_layer, output_layer)

c = optimizers.adam(lr = 0.01)
model1.compile(loss="binary_crossentropy", optimizer=c, metrics=["accuracy"])

W0809 11:29:10.470212 12376 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 11:29:10.488203 12376 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 11:29:10.495196 12376 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 11:29:10.532189 12376 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\

In [0]:
model1.save("unet_101_binary_crossentropy.h5")

In [0]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 101, 101, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 101, 101, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 101, 101, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 25, 25, 32)   0           max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 25, 25, 64)   18496       dropout_2[0][0]                  
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 25, 25, 64)   256         conv2d_11[0][0]                  
__________________________________________________________________________________________________
activation_11 (Activation)      (None, 25, 25, 64)   0           batch_normalization_11[0][0]     
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 25, 25, 64)   36928       activation_11[0][0]              
__________

__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 6, 6, 256)    295168      dropout_4[0][0]                  
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 6, 6, 256)    1024        conv2d_21[0][0]                  
__________________________________________________________________________________________________
activation_21 (Activation)      (None, 6, 6, 256)    0           batch_normalization_21[0][0]     
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 6, 6, 256)    590080      activation_21[0][0]              
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 6, 6, 256)    1024        conv2d_22[0][0]                  
__________

concatenate_2 (Concatenate)     (None, 25, 25, 128)  0           conv2d_transpose_2[0][0]         
                                                                 activation_15[0][0]              
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 25, 25, 128)  0           concatenate_2[0][0]              
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 25, 25, 64)   73792       dropout_6[0][0]                  
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 25, 25, 64)   256         conv2d_31[0][0]                  
__________________________________________________________________________________________________
activation_31 (Activation)      (None, 25, 25, 64)   0           batch_normalization_31[0][0]     
__________

activation_40 (Activation)      (None, 50, 50, 32)   0           batch_normalization_40[0][0]     
__________________________________________________________________________________________________
conv2d_transpose_4 (Conv2DTrans (None, 101, 101, 16) 4624        activation_40[0][0]              
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 101, 101, 32) 0           conv2d_transpose_4[0][0]         
                                                                 activation_5[0][0]               
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 101, 101, 32) 0           concatenate_4[0][0]              
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 101, 101, 16) 4624        dropout_8[0][0]                  
__________

In [0]:
num_epochs = 100
to_be_saved = "models/model_101.h5"                  # <---- change here the name of the file to be saved

early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint(to_be_saved, save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

batch_size = 32

In [0]:
history = model1.fit(x_train,
                     y_train,
                     validation_data=[x_valid,
                                      y_valid],
                     epochs=num_epochs,
                     batch_size=batch_size,
                     callbacks=[early_stopping, 
                               model_checkpoint,
                               reduce_lr]
                    )

KeyboardInterrupt: 